# API

> Module to get data

In [1]:
#| default_exp api

In [1]:
# | exporti

import datetime
from datetime import datetime as dt
import pandas as pd
import altair as alt
from altair_data_server import data_server
alt.data_transformers.enable('data_server')
%load_ext autoreload
%autoreload 2
from entsoe import EntsoePandasClient
from ents import db


In [2]:
def plot_heatmap(df):
    chart = alt.Chart(df).mark_area().encode(
    x="Timestamp:T",
    y="Generation:Q",
    color="Type:N",
    tooltip="Type:N"
)
    chart.display()

In [3]:
# | export
def process_df(df):
    df = df.drop('Actual Consumption', axis=1, level=1)
    df.reset_index(inplace=True, names = 'Timestamp')
    df = df.droplevel(1, axis=1)
    df = df.melt('Timestamp', var_name='Type', value_name='Generation')
    return df

In [4]:
#| hide
# https://github.com/EnergieID/entsoe-py
# https://transparency.entsoe.eu/content/static_content/Static%20content/web%20api/Guide.html#_generation_domain
conn = db.create_connection('test.db')
token = open('../token', 'r').read()
client = EntsoePandasClient(api_key=token)

start = pd.Timestamp('20230101', tz='Europe/Brussels')
end = start + pd.Timedelta(days=1)
country_code = 'NL'  # Belgium
df = client.query_generation(country_code, start=start,end=end, psr_type=None, nett=False)

df = process_df(df)
plot_heatmap(df)
df.to_sql('test2.db', conn, if_exists='append')

d:\Programmeren\adventofcode\.aocenvpy311\Lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

960

In [5]:
import time

def timeit(f, *args, **kwargs):
    def func(*args, **kwargs):
        t0 = time.time()
        res = f(*args, **kwargs)
        print('timed', time.time()-t0)
        return res
    return func

@timeit
def _cut_into_periods(start, end, amount_periods = 3):
    
    # function takes a start and end datetime. Determines the number of rows and splits that into periods of SPLIT_ROWCOUNT
    delta = (end - start) // amount_periods

    cur = start
    while cur < end:
        start = cur
        cur += delta
        yield(start, min(cur, end))

In [6]:
from entsoe.parsers import parse_prices, parse_loads, parse_generation, \
    parse_installed_capacity_per_plant, parse_crossborder_flows, \
    parse_unavailabilities, parse_contracted_reserve, parse_imbalance_prices_zip, \
    parse_imbalance_volumes_zip, parse_netpositions, parse_procured_balancing_capacity, \
    parse_water_hydro

In [7]:
# lxml & text = 6.6
# html parser & text = 7.4
# html parser & content = 7.1
# lxml & content = 6.0

# with soupstrainer
# lxml & text = 6.5
# html parser & text = 7.4
# html parser & content = 7.1
# lxml & content = 5.9

In [8]:
import pytz
import requests
def _datetime_to_str(dtm: pd.Timestamp) -> str:
    """
    Convert a datetime object to a string in UTC
    of the form YYYYMMDDhh00

    Parameters
    ----------
    dtm : pd.Timestamp
        Recommended to use a timezone-aware object!
        If timezone-naive, UTC is assumed

    Returns
    -------
    str
    """
    if dtm.tzinfo is not None and dtm.tzinfo != pytz.UTC:
        dtm = dtm.tz_convert("UTC")
    fmt = '%Y%m%d%H00'
    ret_str = dtm.round(freq='h').strftime(fmt)
    return ret_str
    
start = pd.Timestamp('20221101', tz='Europe/Brussels')
end = start + pd.Timedelta(days=31)
for s,e in _cut_into_periods(start, end, 1):
    st = _datetime_to_str(s)
    en = _datetime_to_str(e)
    response =  requests.get('https://web-api.tp.entsoe.eu/api', params = {'documentType': 'A75', 'processType': 'A16', 'in_Domain': '10YNL----------L', 'securityToken': '09d5b83a-0730-40e4-a48b-62c6e22cb0a6', 'periodStart': st, 'periodEnd': en})
    text = response.text
    

timed 0.0


In [9]:
from lxml import etree
from io import BytesIO
xml = response.content
event, element  = next(etree.iterparse(BytesIO(xml), tag='{*}TimeSeries'))
names = set()
for e in element.iter():
    names.add(e.tag)
names

{'{urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0}MktPSRType',
 '{urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0}Period',
 '{urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0}Point',
 '{urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0}TimeSeries',
 '{urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0}businessType',
 '{urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0}curveType',
 '{urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0}end',
 '{urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0}inBiddingZone_Domain.mRID',
 '{urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0}mRID',
 '{urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0}objectAggregation',
 '{urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0}position',
 '{urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0}psrType',
 '{urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0}quantity',
 '{urn:iec62325.351:tc57wg16:45

In [10]:

a = parse_generation(response.content, nett=False)

In [11]:
a

Biomass                     \
                          Actual Aggregated Actual Consumption   
2022-10-31 23:00:00+00:00              19.0                0.0   
2022-10-31 23:15:00+00:00              19.0                0.0   
2022-10-31 23:30:00+00:00              19.0                0.0   
2022-10-31 23:45:00+00:00              19.0                0.0   
2022-11-01 00:00:00+00:00              20.0                0.0   
...                                     ...                ...   
2022-12-01 21:45:00+00:00              22.0                0.0   
2022-12-01 22:00:00+00:00              27.0                0.0   
2022-12-01 22:15:00+00:00              24.0                0.0   
2022-12-01 22:30:00+00:00              22.0                0.0   
2022-12-01 22:45:00+00:00              22.0                0.0   

                                 Fossil Gas                     \
                          Actual Aggregated Actual Consumption   
2022-10-31 23:00:00+00:00            3135.0              218.0   
2022-10-31 23:15:00+00:00            2866.0              224.0   
2022-10-31 23:30:00+00:00            2584.0              231.0   
2022-10-31 23:45:00+00:00            2443.0              238.0   
2022-11-01 00:00:00+00:00            1801.0              203.0   
...                                     ...                ...   
2022-12-01 21:45:00+00:00            9435.0              261.0   
2022-12-01 22:00:00+00:00            9390.0              248.0   
2022-12-01 22:15:00+00:00            9216.0              256.0   
2022-12-01 22:30:00+00:00            9139.0              259.0   
2022-12-01 22:45:00+00:00            8959.0              260.0   

                           Fossil Hard coal                     \
                          Actual Aggregated Actual Consumption   
2022-10-31 23:00:00+00:00             879.0               16.0   
2022-10-31 23:15:00+00:00             852.0               15.0   
2022-10-31 23:30:00+00:00             823.0               15.0   
2022-10-31 23:45:00+00:00             809.0               16.0   
2022-11-01 00:00:00+00:00             817.0               16.0   
...                                     ...                ...   
2022-12-01 21:45:00+00:00            2085.0                6.0   
2022-12-01 22:00:00+00:00            2084.0               13.0   
2022-12-01 22:15:00+00:00            2095.0               13.0   
2022-12-01 22:30:00+00:00            2082.0               13.0   
2022-12-01 22:45:00+00:00            2095.0               13.0   

                          Hydro Run-of-river and poundage                     \
                                        Actual Aggregated Actual Consumption   
2022-10-31 23:00:00+00:00                             0.0                0.0   
2022-10-31 23:15:00+00:00                             0.0                0.0   
2022-10-31 23:30:00+00:00                             0.0                0.0   
2022-10-31 23:45:00+00:00                             0.0                0.0   
2022-11-01 00:00:00+00:00                             0.0                0.0   
...                                                   ...                ...   
2022-12-01 21:45:00+00:00                             0.0                0.0   
2022-12-01 22:00:00+00:00                             0.0                0.0   
2022-12-01 22:15:00+00:00                             0.0                0.0   
2022-12-01 22:30:00+00:00                             0.0                0.0   
2022-12-01 22:45:00+00:00                             0.0                0.0   

                                    Nuclear                     \
                          Actual Aggregated Actual Consumption   
2022-10-31 23:00:00+00:00             479.0                0.0   
2022-10-31 23:15:00+00:00             479.0                0.0   
2022-10-31 23:30:00+00:00             479.0                0.0   
2022-10-31 23:45:00+00:00             479.0                0.0   
2022-11-01 00:00:00+00

In [12]:
import asyncio
import requests
import time
from functools import partial
import urllib3

http = urllib3.PoolManager(num_pools=50)
@timeit
def sync_method(s,e):
    """Some sync function"""
    kwargs = {'country_code' : 'NL', 'psr_type':None, 'nett':False}
    res = client.query_generation(start=s,end=e, **kwargs)
    # response =  requests.get('https://web-api.tp.entsoe.eu/api', params = {'documentType': 'A75', 'processType': 'A16', 'in_Domain': '10YNL----------L', 'securityToken': '09d5b83a-0730-40e4-a48b-62c6e22cb0a6', 'periodStart': st, 'periodEnd': en})
    # df = parse_generation(response.content, nett=False)
    return res

loop = asyncio.get_event_loop()

async def run_sync_in_executor(s,e, pool=None):
    """Wrapper around run in executor"""
    # run_in_executor should be awaited, otherwise run_in_executor
    # just returns coroutine (not its result!)
    
    value = await loop.run_in_executor(pool, sync_method, s,e)
    return value

async def main(start, end):
    t0 = time.time()
    loop = asyncio.get_running_loop()
    # https://stackoverflow.com/questions/41063331/how-to-use-asyncio-with-existing-blocking-library
    output = await asyncio.gather(*[run_sync_in_executor(s,e) for s,e in _cut_into_periods(start,end,1)])
    print('total time', time.time()-t0)
    return output

start = pd.Timestamp('20220101', tz='Europe/Brussels')
end = start + pd.Timedelta(days=365)

res = await main(start, end)
print(sum([len(p) for p in res]))

timed 0.0
timed 117.14849066734314
total time 117.15319728851318
35040


In [ ]:
# 1 month generation
no change
async 1: 27.8 / 10?
async 10: 25
async 30: 31
with regex
async 1: 5
async 10: 3.9
async 30: 4.8
with lxml
async 1: 4.4
async 10: 2.3
async 30: 3.2


In [ ]:
# 1 year generation
no change
async 1: 382 / 117
async 6:    /  120
async 10: 120
async 30: 
with change
async 1: 119
async 10: 37
async 30: 38
with lxml
async 1: 34
async 10: 18

In [ ]:
async def main():
    print(1)
    
await main()

1
